In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import os
import numpy as np
import cv2

In [ ]:
DATASET_DIR = "Dataset/Event_clips"
BATCH_SIZE = 8
IMG_SIZE = (224, 224)
SEQ_LENGTH = 16  # Number of frames per video clip

model_id = "a0"  # Choose from a0 to a6
movinet_model = hub.load(f"https://tfhub.dev/tensorflow/movinet/{model_id}/stream/kinetics-600/classification/3")

In [ ]:
def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, IMG_SIZE)
        frame = frame / 255.0  
        frames.append(frame)
    
    cap.release()
    
    indices = np.linspace(0, len(frames) - 1, SEQ_LENGTH, dtype=int)
    sampled_frames = np.array([frames[i] for i in indices])
    return sampled_frames


In [ ]:
def load_dataset(data_dir):
    """Loads videos and corresponding labels."""
    classes = sorted(os.listdir(data_dir))
    class_to_idx = {cls: i for i, cls in enumerate(classes)}
    
    video_paths = []
    labels = []
    
    for cls in classes:
        cls_path = os.path.join(data_dir, cls)
        for video_file in os.listdir(cls_path):
            video_paths.append(os.path.join(cls_path, video_file))
            labels.append(class_to_idx[cls])
    
    return video_paths, labels, class_to_idx

video_paths, labels, class_to_idx = load_dataset(DATASET_DIR)
num_classes = len(class_to_idx)

In [ ]:
def create_tf_dataset(video_paths, labels):
    """Creates a TensorFlow dataset from video paths and labels."""
    def load_video(path, label):
        frames = preprocess_video(path.numpy().decode("utf-8"))
        return frames, label
    
    dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
    dataset = dataset.map(lambda x, y: tf.py_function(load_video, [x, y], [tf.float32, tf.int32]))
    dataset = dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

train_ds = create_tf_dataset(video_paths, labels)

In [ ]:
inputs = tf.keras.layers.Input(shape=(SEQ_LENGTH, 224, 224, 3))
x = movinet_model.signatures["serving_default"](inputs)
x = tf.keras.layers.Dense(num_classes, activation='softmax')(x["logits"])

model = tf.keras.Model(inputs, x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the Model
model.fit(train_ds, epochs=10)

# Save Model
model.save("movinet_football_highlight.h5")
